# Data normalisation pipeline


In [67]:
import pandas as pd
import numpy as np
import joblib
import os

behavData = 'C:\\Users\\Hao-Ting\\Documents\\Work\\Project_CCA\\sourcedata\\CS_MWQafterRS.csv'
data = np.genfromtxt(behavData, delimiter=',')[1:,2:]
labels = (pd.read_csv(behavData).columns).tolist()[2:]

In [38]:
print labels

['Future', 'Past', 'Self', 'Other', 'Pos', 'Neg', 'Words', 'Images', 'Evolving', 'Habit', 'Detailed', 'Vivid', 'Deliberate', 'Spontaneous', 'Abstract', 'Creative', 'Decoupling', 'Important', 'Intrusive', 'Problem', 'Realistic', 'Stop', 'Thematic', 'HereNow', 'TemporalDistance']


In [68]:
excludeIdx = []

#exclde cases with more than 20 nan
excludeIdx = []
for i in range(data.shape[0]):
    n = np.count_nonzero(np.isnan(data)[i])
    if n>20:
        excludeIdx.append(i)

excludeIdx = np.array(excludeIdx)
#exclude the participants
x = np.delete(data, excludeIdx, 0)
data = x[:,:]

In [71]:
print excludeIdx
print data.shape

[121]
(164L, 25L)


In [70]:
S = data.sum(axis=0) / data.shape[0]
data -= S[np.newaxis, :]
var = (data ** 2).sum(axis=0)
var[var == 0] = 1
data /= var

In [72]:
np.save('MWQ_afterRS_zscore_exludeNaN_ind22', data) #save for later

In [106]:
import glob, os, sys
import numpy as np
from nilearn.image import resample_img, index_img
import nibabel as nib
from nilearn.input_data import NiftiLabelsMasker

roiLabel = 'C:\\Users\\Hao-Ting\\Documents\\Work\\tcorr05_2level_all.nii.gz'
all_label_atlas_nii = nib.load(roiLabel)

## Get the mask I need from a set of atals selections


In [129]:
label_atlas = all_label_atlas_nii.get_data()[..., 8] 
type(label_atlas)
print label_atlas.shape
print all_label_atlas_nii.get_qform()
print np.min(label_atlas)
print np.max(label_atlas)
TEMP_rois = nib.Nifti1Image(
            label_atlas,
            affine=all_label_atlas_nii.get_qform(),
            header=all_label_atlas_nii.get_header(),
        )

(47L, 56L, 46L)
[[  -4.    0.    0.   92.]
 [   0.    4.    0. -128.]
 [   0.    0.    4.  -72.]
 [   0.    0.    0.    1.]]
0
90


## Get the spatial information of my scanning data


In [130]:
rs_niis = sorted(glob.glob('D:\\CS_brain_preprocessed' + os.sep + '*.nii.gz')) #windows
tmp_nii_path = rs_niis[0]
tmp_nii = nib.load(tmp_nii_path)
print tmp_nii.get_qform()
print tmp_nii.shape

[[  -2.    0.    0.   90.]
 [   0.    2.    0. -126.]
 [   0.    0.    2.  -72.]
 [   0.    0.    0.    1.]]
(91, 109, 91, 180)


## Resample the ROI mask

In [131]:
re_rois = resample_img(
            img=TEMP_rois,
            target_affine=tmp_nii.get_affine(),
            target_shape=tmp_nii.shape[:3],
            interpolation='nearest')
# nib.save(re_rois, 'craddock2011_90_ROI') #save for later

In [132]:
masker = NiftiLabelsMasker(labels_img=re_rois, standardize=True,
                           memory='nilearn_cache', verbose=0)
masker.fit()

NiftiLabelsMasker(background_label=0, detrend=False, high_pass=None,
         labels_img=<nibabel.nifti1.Nifti1Image object at 0x00000000088309E8>,
         low_pass=None, mask_img=None, memory='nilearn_cache',
         memory_level=1, resampling_target='data', smoothing_fwhm=None,
         standardize=True, t_r=None, verbose=0)

In [134]:
40**2/2-40

760

In [ ]:
for i_rs_img, rs_img in enumerate(rs_niis):
    print('%i/%i: %s' % (i_rs_img + 1, len(rs_niis), rs_img))
    rs_reg_ts = masker.transform(rs_img)
    corr_mat = np.corrcoef(rs_reg_ts.T)
    triu_inds = np.triu_indices(corr_mat.shape[0], 1)
    corr_mat_vect = corr_mat[triu_inds]
    # save for later
    corr_mat_vect_list.append(corr_mat_vect)
corr_mat_vect_array = np.array(corr_mat_vect_list)
print(corr_mat_vect_array.shape) 